In [ ]:
# import libraries
%load_ext autoreload
%reload_ext autoreload
%autoreload 2

import numpy as np
import matplotlib.pyplot as plt

from sklearn.svm import SVR
from sklearn.gaussian_process.kernels import ExpSineSquared
from sklearn.kernel_ridge import KernelRidge
from sklearn.gaussian_process import GaussianProcessRegressor
from sklearn.gaussian_process.kernels import WhiteKernel, RBF, RationalQuadratic
from sklearn.preprocessing import RobustScaler, QuantileTransformer, PowerTransformer, MaxAbsScaler, StandardScaler, KBinsDiscretizer

In [6]:
prediction = np.empty((0,1))

In [7]:
# import data
raw = np.loadtxt('../1/o.csv', skiprows=1, delimiter=',')
# preprocess data
y_train = raw[:,0:1]
y_train = raw[:,1:2]
#transformer = RobustScaler(quantile_range=(0.1, 0.9))
transformer = StandardScaler().fit(y_train) # standardization
y_train = transformer.transform(y_train)

x_train = np.array([[i] for i in range(len(y_train))])

X = np.array([[i] for i in range(48)])

# define kernel
long_term_trend_kernel = 40.0**2 * RBF(length_scale=40) 
#long_term_trend_kernel = 40.0**2 * ExpSineSquared(length_scale=40, periodicity=12.0, periodicity_bounds=(12, 36)) *  40.0**2 *RBF(length_scale=40) 
seasonal_kernel = (
    12.0**2
    #* RBF(length_scale=10)
    * ExpSineSquared(length_scale=0.1, periodicity=12.0, periodicity_bounds=(12, 12))
)
irregularities_kernel = 3**2 * RationalQuadratic(length_scale=0.1, alpha=1.0)
noise_kernel = 1**2 * RBF(length_scale=0.1) + WhiteKernel(noise_level_bounds=(0, 1))
# combine kernel
kernel = long_term_trend_kernel + seasonal_kernel + irregularities_kernel + noise_kernel
# initialize models
gaussian_process = GaussianProcessRegressor(kernel=kernel)
kernel_ridge = KernelRidge(kernel=kernel)
# support vector regression
C = 100
svr_rbf = SVR(kernel="rbf", C=C, gamma=0.1, epsilon=0.1)
svr_lin = SVR(kernel="linear", C=C, gamma="auto")
svr_poly = SVR(kernel="poly", C=C, gamma="auto", degree=2, epsilon=0.1, coef0=0)

In [ ]:
# visualize model results
lw = 2
models = [svr_rbf, gaussian_process, kernel_ridge]
kernel_label = ["RBF", "gaussian_process", "kernel_ridge"]
model_color = ["m", "c", "g"]

fig, axes = plt.subplots(nrows=1, ncols=3, figsize=(15, 10), sharey=True)
for ix, svr in enumerate(models):
    y = svr.fit(x_train, y_train).predict(X)
    axes[ix].plot(
        X,
        y,
        color=model_color[ix],
        lw=lw,
        label="{} model".format(kernel_label[ix]),
    )
    axes[ix].scatter(
        X[:],
        y[:],
        facecolor="none",
        edgecolor=model_color[ix],
        s=50,
        label="{} support vectors".format(kernel_label[ix]),
    )
    axes[ix].scatter(
        x_train,
        y_train,
        facecolor="none",
        edgecolor="k",
        s=50,
        label="training data",
    )
    axes[ix].legend(
        loc="upper center",
        bbox_to_anchor=(0.5, 1.1),
        ncol=1,
        fancybox=True,
        shadow=True,
    )

fig.text(0.5, 0.04, "data", ha="center", va="center")
fig.text(0.06, 0.5, "target", ha="center", va="center", rotation="vertical")
fig.suptitle("Support Vector Regression", fontsize=14)
plt.show()
#print(gaussian_process.kernel_)

In [ ]:
X = np.array([[i] for i in range(39, 48)])
predictor = kernel_ridge
print("Predicted Y: ",predictor.fit(x_train, y_train).predict(X))
temp = transformer.inverse_transform(predictor.predict(X).reshape(-1, 1))
prediction = np.concatenate((prediction, temp))
print("Predicted Raw: ", prediction)
print(prediction.shape)